In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os 
os.getcwd()

'C:\\Users\\jsoto\\rmp-spatial-join\\notebooks'

In [2]:
# Path to data file
# Note that you may need to add an extra slash ("\") in front of the existing slashes to avoid errors
data_path = Path("C:\\Users\\jsoto\\DOI\\BGC Projects (v3) - Documents\\Mapping Data Workflow\\Mapping Directory for Spatial Join and Statistics\\Toe Pulse 2018\\Data\\Spatial Join\\2018-10-04_fts_20secMed_spatialJoin.csv")

df_data = pd.read_csv(data_path)
df_data.head()


,FTS Timestamp,Latitude,Longitude,WS_fDOM_QSU,WS_fCHLA_ug/L,SUNANO3uM,CStar_Tr,CStar_c,TSG_Temp,TSG_Cond,...,EXODOmgL,EXODOSAT,EXOTurbFNU,EXOfDOM_QSU,EXOfChla_ug/L,index_right,Location,Shape_Leng,Shape_Area,Poly_ID
0,10/4/2018 8:29,38.15562,-121.682660,20.605298,1.504000,14.345,0.729710,3.151085,19.46790,127.4,...,8.600,93.635,2.81,14.297678,NaN,4410.0,Sacramento River,0.019328,0.000012,A677
1,10/4/2018 8:29,38.15562,-121.682655,20.603970,1.505902,14.340,0.729709,3.151097,19.46780,127.4,...,8.600,93.600,2.81,14.316774,NaN,4410.0,Sacramento River,0.019328,0.000012,A677
2,10/4/2018 8:29,38.15562,-121.682650,20.604907,1.509847,14.345,0.729550,3.153269,19.46770,127.4,...,8.595,93.570,2.81,14.327609,NaN,4410.0,Sacramento River,0.019328,0.000012,A677
3,10/4/2018 8:29,38.15562,-121.682650,20.603970,1.513792,14.340,0.729392,3.155441,19.46760,127.4,...,8.590,93.540,2.81,14.338444,NaN,4410.0,Sacramento River,0.019328,0.000012,A677
4,10/4/2018 8:30,38.15562,-121.682650,20.602493,1.524879,14.340,0.729247,3.157426,19.46755,127.4,...,8.590,93.520,2.81,14.344508,NaN,4410.0,Sacramento River,0.019328,0.000012,A677


In [3]:
# First lets see what columns we have
df_data.columns.values

array(['FTS Timestamp', 'Latitude', 'Longitude', 'WS_fDOM_QSU',
       'WS_fCHLA_ug/L', 'SUNANO3uM', 'CStar_Tr', 'CStar_c', 'TSG_Temp',
       'TSG_Cond', 'TSG_Salinity', 'EXOTemp', 'EXOSpCond', 'EXOpH',
       'EXOpHmV', 'EXODOmgL', 'EXODOSAT', 'EXOTurbFNU', 'EXOfDOM_QSU',
       'EXOfChla_ug/L', 'index_right', 'Location', 'Shape_Leng',
       'Shape_Area', 'Poly_ID'], dtype=object)

In [4]:
# Make a variable for the polygon id column name
# Change this to the name you identified in the list above
poly_id_col = "Poly_ID"

print(len(df_data))
df_data = df_data.dropna(subset=[poly_id_col])
print(len(df_data))

9432
9415


In [5]:
# Now lets make a list of the ones we don't want to include
# I am just copy/pasting non-constituent columns from above into this list
cols_to_drop = [
    'FTS Timestamp', 'Latitude', 'Longitude', 'index_right',
       'Location', 'Shape_Leng', 'Shape_Area'
]

In [6]:
df_data = df_data.drop(cols_to_drop, axis="columns")
df_data.columns.values

array(['WS_fDOM_QSU', 'WS_fCHLA_ug/L', 'SUNANO3uM', 'CStar_Tr', 'CStar_c',
       'TSG_Temp', 'TSG_Cond', 'TSG_Salinity', 'EXOTemp', 'EXOSpCond',
       'EXOpH', 'EXOpHmV', 'EXODOmgL', 'EXODOSAT', 'EXOTurbFNU',
       'EXOfDOM_QSU', 'EXOfChla_ug/L', 'Poly_ID'], dtype=object)

In [7]:
# Compute Stats
# First group the data by poly_id_col
df_grouped = df_data.groupby(poly_id_col)

In [8]:
# Now lets define the statistics we want to compute in a list that we can pass to the pandas aggregation function
# For more information on what can go into this list check out: 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html
#KO: added std to stats list 5/19/2020

stats = [
    "min",
    "max",
    "mean",
    "median",
    "std"
]

# Compute the statistics defined above for each polygon
df_stats = df_grouped.agg(stats)

In [9]:
df_stats

WS_fDOM_QSU                                            WS_fCHLA_ug/L  \
                min        max       mean     median       std           min   
Poly_ID                                                                        
A2164     42.280643  42.660175  42.450990  42.449421  0.110879      3.040422   
A2165     42.214697  42.618949  42.410930  42.463914  0.113947      3.019770   
A2166     41.745960  42.629073  42.278960  42.285884  0.296781      2.922775   
A2167     41.462989  42.202588  41.777472  41.744369  0.202732      2.929127   
A2168     41.196586  41.761994  41.452493  41.426408  0.148264      2.937644   
...             ...        ...        ...        ...       ...           ...   
A679      19.972667  20.058244  20.037141  20.047143  0.025726      1.718853   
A680      20.048788  20.063099  20.053264  20.053049  0.003891      1.766292   
A681      20.053507  20.065694  20.057962  20.055319  0.004827      1.801392   
A682      20.037753  20.075850  20.063405  20.071400  0.014873      1.827188   
A683      19.991005  20.333761  20.201867  20.265358  0.137873      1.830963   

                                                 ... EXOfDOM_QSU             \
              max      mean    median       std  ...         min        max   
Poly_ID                                          ...                          
A2164    3.151407  3.087750  3.078137  0.030868  ...   37.367113  38.593279   
A2165    3.104216  3.055457  3.047588  0.026076  ...   37.441866  37.943286   
A2166    3.082032  2.996687  2.980622  0.056211  ...   37.395944  38.458678   
A2167    3.007796  2.971451  2.958794  0.026001  ...   37.137171  38.458678   
A2168    3.012976  2.983442  2.999163  0.027791  ...   36.703972  37.933629   
...           ...       ...       ...       ...  ...         ...        ...   
A679     1.766292  1.731063  1.723792  0.015389  ...   18.850073  18.889801   
A680     1.800340  1.786832  1.786300  0.010987  ...   18.844022  18.882062   
A681     1.825710  1.809706  1.804620  0.009173  ...   18.872906  18.882062   
A682     1.839051  1.831701  1.831355  0.002580  ...   18.878521  18.892819   
A683     1.912725  1.877399  1.892813  0.027932  ...   18.889988  19.234584   

                                        EXOfChla_ug/L                      \
              mean     median       std           min       max      mean   
Poly_ID                                                                     
A2164    37.940689  37.870948  0.379571      0.568319  0.693441  0.625336   
A2165    37.763869  37.780642  0.134194      0.471925  0.664356  0.597725   
A2166    37.690742  37.732654  0.272085      0.481510  0.625842  0.592909   
A2167    37.671170  37.333874  0.497015      0.577534  0.616184  0.596795   
A2168    37.274265  37.127387  0.389078      0.572693  0.616125  0.591361   
...            ...        ...       ...           ...       ...       ...   
A679     18.872256  18.876776  0.011434           NaN       NaN       NaN   
A680     18.866997  18.872906  0.013043           NaN       NaN       NaN   
A681     18.879105  18.878804  0.002251           NaN       NaN       NaN   
A682     18.887509  18.891120  0.005616           NaN       NaN       NaN   
A683     19.060480  19.135151  0.134901      0.000000  0.028504  0.007255   

                             
           median       std  
Poly_ID                      
A2164    0.626023  0.038549  
A2165    0.625722  0.063153  
A2166    0.606411  0.032467  
A2167    0.597015  0.011581  
A2168    0.587273  0.012583  
...           ...       ...  
A679          NaN       NaN  
A680          NaN       NaN  
A681          NaN       NaN  
A682          NaN       NaN  
A683     0.000000  0.011450  

[296 rows x 85 columns]

In [10]:
# Flatten the hierarchical columns
df_stats.columns = [' '.join(col).strip() for col in df_stats.columns.values]

In [11]:
df_stats.head()

,WS_fDOM_QSU min,WS_fDOM_QSU max,WS_fDOM_QSU mean,WS_fDOM_QSU median,WS_fDOM_QSU std,WS_fCHLA_ug/L min,WS_fCHLA_ug/L max,WS_fCHLA_ug/L mean,WS_fCHLA_ug/L median,WS_fCHLA_ug/L std,...,EXOfDOM_QSU min,EXOfDOM_QSU max,EXOfDOM_QSU mean,EXOfDOM_QSU median,EXOfDOM_QSU std,EXOfChla_ug/L min,EXOfChla_ug/L max,EXOfChla_ug/L mean,EXOfChla_ug/L median,EXOfChla_ug/L std
Poly_ID,,,,,,,,,,,,,,,,,,,,,
A2164,42.280643,42.660175,42.450990,42.449421,0.110879,3.040422,3.151407,3.087750,3.078137,0.030868,...,37.367113,38.593279,37.940689,37.870948,0.379571,0.568319,0.693441,0.625336,0.626023,0.038549
A2165,42.214697,42.618949,42.410930,42.463914,0.113947,3.019770,3.104216,3.055457,3.047588,0.026076,...,37.441866,37.943286,37.763869,37.780642,0.134194,0.471925,0.664356,0.597725,0.625722,0.063153
A2166,41.745960,42.629073,42.278960,42.285884,0.296781,2.922775,3.082032,2.996687,2.980622,0.056211,...,37.395944,38.458678,37.690742,37.732654,0.272085,0.481510,0.625842,0.592909,0.606411,0.032467
A2167,41.462989,42.202588,41.777472,41.744369,0.202732,2.929127,3.007796,2.971451,2.958794,0.026001,...,37.137171,38.458678,37.671170,37.333874,0.497015,0.577534,0.616184,0.596795,0.597015,0.011581
A2168,41.196586,41.761994,41.452493,41.426408,0.148264,2.937644,3.012976,2.983442,2.999163,0.027791,...,36.703972,37.933629,37.274265,37.127387,0.389078,0.572693,0.616125,0.591361,0.587273,0.012583


In [12]:
# run the loop desired number of times. final output file will have the stats based on the final loop. 
# adjust number of iterations to make sure values converge using printed stat below
num_iterations = 12
tmp_df_data = df_data.copy() #copy of original df_data to preserve data
tmp_df_stats = df_stats.copy() 
for num in range(num_iterations):
    
    # merge original data and stats dataframes on poly ID
    df_merged = df_data.merge(tmp_df_stats, how = 'outer', left_on = 'Poly_ID', right_on = 'Poly_ID', )
    
    convergence = 0
    for col in df_data.columns:
        # skip location, poly_id, lat, long columns when filtering data
        if col in ('Location','Poly_ID','Latitude','Longitude'):
            continue 

        # creates series to hold mean and standard deviation values
        col_mean = col + ' mean' 
        col_std = col + ' std' # standard deviation

        # calculates and creates a column for 2x std
        col_2xstd = 2*df_merged[col_std]

        # calculates the difference between the instantaneous value and the mean value of its corresponding polygon
        col_diff_mean = abs(df_merged[col] - df_merged[col_mean])

        # if the difference between the instantenous value and the mean is greater than 2xstd replace the value with a NAN
        remove_mask = col_diff_mean > col_2xstd
        df_merged[col][remove_mask] = np.nan   
        convergence += col_2xstd.sum()
    print("Sum of standard deviations converging on: ", convergence)

    # creates dataframe, df_data_filtered
    # dropping extra columns created during filtering
    # filtered data retained, and columns kept are based on columns that existed in df_data
    tmp_df_data = df_merged[df_data.columns]
    
    # Re-run stats on filtered data
    df_grouped_filtered = tmp_df_data.groupby(poly_id_col)

    # Compute the statistics defined above for each polygon
    tmp_df_stats = df_grouped_filtered.agg(stats)
    
    # Flatten the hierarchical columns
    tmp_df_stats.columns = [' '.join(col).strip() for col in tmp_df_stats.columns.values]

# change name of dataframe for clarity after processing
df_stats_filtered = tmp_df_stats

C:\Users\jsoto\Anaconda3\envs\rmp-spatial-join-stats\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Sum of standard deviations converging on:  194169.70012091252
Sum of standard deviations converging on:  162807.56002930983
Sum of standard deviations converging on:  154326.34823196928
Sum of standard deviations converging on:  149518.32289899004
Sum of standard deviations converging on:  146507.3594203793
Sum of standard deviations converging on:  144924.38801379502
Sum of standard deviations converging on:  144163.78425121625
Sum of standard deviations converging on:  143682.12211611963
Sum of standard deviations converging on:  143479.71012407992
Sum of standard deviations converging on:  143325.36335703358
Sum of standard deviations converging on:  143191.29028805584
Sum of standard deviations converging on:  142962.24485037566


In [13]:
tmp_df_stats

,WS_fDOM_QSU min,WS_fDOM_QSU max,WS_fDOM_QSU mean,WS_fDOM_QSU median,WS_fDOM_QSU std,WS_fCHLA_ug/L min,WS_fCHLA_ug/L max,WS_fCHLA_ug/L mean,WS_fCHLA_ug/L median,WS_fCHLA_ug/L std,...,EXOfDOM_QSU min,EXOfDOM_QSU max,EXOfDOM_QSU mean,EXOfDOM_QSU median,EXOfDOM_QSU std,EXOfChla_ug/L min,EXOfChla_ug/L max,EXOfChla_ug/L mean,EXOfChla_ug/L median,EXOfChla_ug/L std
Poly_ID,,,,,,,,,,,,,,,,,,,,,
A2164,42.280643,42.660175,42.450990,42.449421,0.110879,3.059304,3.100071,3.077374,3.076940,0.011725,...,37.367113,38.593279,37.940689,37.870948,0.379571,0.568319,0.693441,0.625336,0.626023,0.038549
A2165,42.214697,42.618949,42.410930,42.463914,0.113947,3.019770,3.104216,3.055457,3.047588,0.026076,...,37.589809,37.943286,37.782832,37.783962,0.112940,0.471925,0.664356,0.597725,0.625722,0.063153
A2166,41.745960,42.629073,42.278960,42.285884,0.296781,2.922775,3.082032,2.996687,2.980622,0.056211,...,37.395944,37.882003,37.608677,37.579916,0.158736,0.572693,0.625842,0.601774,0.606469,0.014653
A2167,41.462989,42.062520,41.741881,41.735349,0.172139,2.929127,3.007796,2.971451,2.958794,0.026001,...,37.137171,38.458678,37.671170,37.333874,0.497015,0.577534,0.616184,0.596795,0.597015,0.011581
A2168,41.196586,41.695656,41.446966,41.424964,0.143559,2.937644,3.012976,2.983442,2.999163,0.027791,...,36.703972,37.933629,37.274265,37.127387,0.389078,0.572693,0.616125,0.591361,0.587273,0.012583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A679,20.037321,20.058244,20.048947,20.049060,0.006905,1.718853,1.731541,1.723644,1.722431,0.004819,...,18.850073,18.889801,18.872256,18.876776,0.011434,NaN,NaN,NaN,NaN,NaN
A680,20.048788,20.054127,20.051917,20.053049,0.002005,1.766292,1.800340,1.786832,1.786300,0.010987,...,18.844022,18.882062,18.866997,18.872906,0.013043,NaN,NaN,NaN,NaN,NaN
A681,20.053507,20.065694,20.057962,20.055319,0.004827,1.801392,1.825710,1.809706,1.804620,0.009173,...,18.878521,18.882062,18.879622,18.879393,0.001320,NaN,NaN,NaN,NaN,NaN


In [14]:
# stats for each polygon based on final iteration of above loop
# Write output file
out_dir = "C:\\Users\\jsoto\\DOI\\BGC Projects (v3) - Documents\\Mapping Data Workflow\\Mapping Directory for Spatial Join and Statistics\\Toe Pulse 2018\\Data\\Statistics"
out_fname = data_path.name.split(".")[0] + "_stats.csv"
out_path = Path(out_dir, out_fname)

# Flatten the hierarchical columns
df_stats_filtered.columns = [''.join(col).strip() for col in df_stats_filtered.columns.values]


# Write the csv
df_stats_filtered.to_csv(out_path)


In [15]:
# use a left join to append lat, long values of delta polygon center points
# Path to center point file
# Note that you may need to add an extra slash ("\") in front of the existing slashes to avoid errors
point_path = Path("C:\\Users\\jsoto\\DOI\\BGC Projects (v3) - Documents\\Mapping Data Workflow\\Mapping Directory for Spatial Join and Statistics\\Center Points xlsx\\DeltaCenterPoints_01.xlsx")

df_point = pd.read_excel(point_path)
df_point.head()

,CL_ID,Location,Latitude,Longitude
0,A4636,Old River,37.841704,-121.537978
1,A5615,Latham Slough,37.993362,-121.512708
2,A4093,Hog Slough,38.166201,-121.491400
3,A3245,South Mokelumne River,38.170279,-121.495870
4,A3207,South Mokelumne River,38.209702,-121.489602


In [16]:
merged_left = pd.merge(left=df_stats_filtered, right=df_point, how='left', left_on='Poly_ID', right_on='CL_ID')
merged_left


,WS_fDOM_QSU min,WS_fDOM_QSU max,WS_fDOM_QSU mean,WS_fDOM_QSU median,WS_fDOM_QSU std,WS_fCHLA_ug/L min,WS_fCHLA_ug/L max,WS_fCHLA_ug/L mean,WS_fCHLA_ug/L median,WS_fCHLA_ug/L std,...,EXOfDOM_QSU std,EXOfChla_ug/L min,EXOfChla_ug/L max,EXOfChla_ug/L mean,EXOfChla_ug/L median,EXOfChla_ug/L std,CL_ID,Location,Latitude,Longitude
0,42.280643,42.660175,42.450990,42.449421,0.110879,3.059304,3.100071,3.077374,3.076940,0.011725,...,0.379571,0.568319,0.693441,0.625336,0.626023,0.038549,A2164,Toe Drain Canal,38.353222,-121.643403
1,42.214697,42.618949,42.410930,42.463914,0.113947,3.019770,3.104216,3.055457,3.047588,0.026076,...,0.112940,0.471925,0.664356,0.597725,0.625722,0.063153,A2165,Toe Drain Canal,38.351817,-121.644052
2,41.745960,42.629073,42.278960,42.285884,0.296781,2.922775,3.082032,2.996687,2.980622,0.056211,...,0.158736,0.572693,0.625842,0.601774,0.606469,0.014653,A2166,Toe Drain Canal,38.350478,-121.644629
3,41.462989,42.062520,41.741881,41.735349,0.172139,2.929127,3.007796,2.971451,2.958794,0.026001,...,0.497015,0.577534,0.616184,0.596795,0.597015,0.011581,A2167,Toe Drain Canal,38.349105,-121.645300
4,41.196586,41.695656,41.446966,41.424964,0.143559,2.937644,3.012976,2.983442,2.999163,0.027791,...,0.389078,0.572693,0.616125,0.591361,0.587273,0.012583,A2168,Toe Drain Canal,38.347822,-121.645979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,20.037321,20.058244,20.048947,20.049060,0.006905,1.718853,1.731541,1.723644,1.722431,0.004819,...,0.011434,NaN,NaN,NaN,NaN,NaN,A679,Sacramento River,38.153518,-121.685911
292,20.048788,20.054127,20.051917,20.053049,0.002005,1.766292,1.800340,1.786832,1.786300,0.010987,...,0.013043,NaN,NaN,NaN,NaN,NaN,A680,Sacramento River,38.152093,-121.686369
293,20.053507,20.065694,20.057962,20.055319,0.004827,1.801392,1.825710,1.809706,1.804620,0.009173,...,0.001320,NaN,NaN,NaN,NaN,NaN,A681,Sacramento River,38.150670,-121.686748
294,20.037753,20.075850,20.063405,20.071400,0.014873,1.828869,1.834843,1.831534,1.831355,0.001676,...,0.005616,NaN,NaN,NaN,NaN,NaN,A682,Sacramento River,38.149323,-121.687423


In [17]:
# stats for each polygon based on final iteration of above loop joined with CL_ID
# Write output file
out_dir2 = "C:\\Users\\jsoto\\DOI\\BGC Projects (v3) - Documents\\Mapping Data Workflow\Mapping Directory for Spatial Join and Statistics\\Toe Pulse 2018\\Data\\CL_ID Join"
out_fname2 = data_path.name.split(".")[0] + "_stats_join.csv"
out_path2 = Path(out_dir2, out_fname2)

# Flatten the hierarchical columns
merged_left.columns = [''.join(col).strip() for col in merged_left.columns.values]


# Write the csv
merged_left.to_csv(out_path2)